In [1]:
import scanpy as sc
import cfp
import os
import random
import pickle
import pandas as pd

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
data_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc"

In [3]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/adata_hvg2000_LV.h5ad")

In [4]:
adata

AnnData object with n_obs × n_vars = 9697974 × 2000
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'total_counts_MT', 'pct_counts_MT', 'log1p_total_counts_MT', 'donor', 'cytokine', 'treatment', 'cell_type'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
    layers: 'counts'

In [5]:
adata.X = adata.layers["counts"]
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [6]:
annotations = pd.read_csv("/lustre/groups/ml01/workspace/ten_million/data/data_2024_12_16/new_cell_type_annotations.csv", index_col=0)
adata.obs["cell_type_new"] = annotations["cell_type_new"] 

/tmp/ipykernel_2225788/1232491902.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations = pd.read_csv("/lustre/groups/ml01/workspace/ten_million/data/data_2024_12_16/new_cell_type_annotations.csv", index_col=0)


In [7]:
adata.obs["condition"] = adata.obs["sample"]

In [8]:
adata.obs["is_control"] = adata.obs.apply(lambda x: True if x["treatment"]=="PBS" else False, axis=1)

In [ ]:
cfp.preprocessing.centered_pca(adata, n_comps=30)

In [ ]:
adata.write("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/pbmc_with_pca.h5ad")

For each cytokine select 6 patients randomly. 
Impute each of the 6 patients' GEX profile trained on 0, 1, 2, 3, 4, 5, 6, 7,8,9,10,11 included other patients. This means training 7+6=13 models and 8*6=48 inferred populations for each cytokine.

All in all, this is 90*13=1170 models, and 90*48=4320 inferred populations.

Naming conventions are f"{cytokine_held_out}_{donor_held_out}_{number_of_included_patients}"

In [ ]:
ADSF           218
4-1BBL         218
BAFF           218
Noggin         218
OSM            218
OX40L          218
IFN-epsilon    218
IL-15          218
IFN-beta